<a href="https://colab.research.google.com/github/yjyjy131/ProblemSolving/blob/main/implementation/boj21609_%EC%83%81%EC%96%B4%EC%A4%91%ED%95%99%EA%B5%90_g2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.acmicpc.net/problem/21609

In [ ]:
import copy
from collections import deque


# 크기가 가장 큰 블록 그룹 찾기
def search_block(board):
    original = copy.deepcopy(board)
    go_x = [0, 0, 1, -1]
    go_y = [1, -1, 0, 0]
    max_n = 0

    block_cords = []
    for i in range(n):
        for j in range(n):
            if 5 >= board[i][j] >= 1:
                board = copy.deepcopy(original)
                b_cords = []
                q = deque([])
                b_color = board[i][j]
                if b_color != -1:
                    q.append((i, j))
                    b_cords.append((i, j))
                    board[i][j] = 100

                    while q:
                        x, y = q.popleft()
                        for dx, dy in zip(go_x, go_y):
                            nx = x + dx
                            ny = y + dy

                            if 0 <= nx < n and 0 <= ny < n:
                                if board[nx][ny] != -1 and board[nx][ny] == b_color or board[nx][ny] == 0:
                                    q.append((nx, ny))
                                    b_cords.append((nx, ny))
                                    board[nx][ny] = 100

                    b_cnt = len(b_cords)
                    if b_cnt >= 2:
                        if max_n == b_cnt:
                            block_cords.append(b_cords)
                        elif max_n < b_cnt:
                            max_n = b_cnt
                            block_cords = [b_cords]

    return block_cords


# 무지개 블록 수가 가장 많은 경우 찾기
def get_rainbow(b_cans):
    max_rb = 0
    rb_cans = []
    for bc in b_cans:
        cnt = 0
        for b in bc:
            if board[b[0]][b[1]] == 0:
                cnt += 1
        if max_rb == cnt:
            rb_cans.append(bc)
        elif max_rb < cnt:
            max_rb = cnt
            rb_cans = [bc]

    return rb_cans


# 기준 블록 찾기
def get_pivot(b_cans):
    p_cans = [b for b in b_cans if board[b[0]][b[1]] != 0]
    if p_cans:
        p_cans = sorted(p_cans)
        return p_cans[0]
    else:
        return None


# 기준 블록의 행이 가장 큰 것
def big_row_block(b_cans):
    p_tmp = 0
    r_cans = []
    for bc in b_cans:
        p = get_pivot(bc)
        if p is not None:
            if p_tmp == p[0]:
                r_cans.append(bc)
            elif p_tmp < p[0]:
                p_tmp = p[0]
                r_cans = [bc]

    return r_cans


# 기준 블록의 열이 가장 큰 것
def big_col_block(b_cans):
    p_tmp = 0
    p_cans = []
    for bc in b_cans:
        p = get_pivot(bc)
        if p is not None:
            if p_tmp == p[1]:
                p_cans.append(bc)
            elif p_tmp < p[1]:
                p_tmp = p[1]
                p_cans = [bc]

    return p_cans


# 블록 그룹의 모든 블록을 제거
def remove_block(blocks):
    s = 0
    for b in blocks:
        s += 1
        board[b[0]][b[1]] = 100

    return pow(s, 2)


# 격자에 중력이 작용
def gravity():
    for _ in range(n):
        for j in range(n):
            for i in range(n-1, -1, -1):
                if board[i][j] != -1 and board[i][j] != 100:
                    nx = i + 1
                    if 0 <= nx < n and board[nx][j] == 100 and board[nx][j] != -1:
                        board[nx][j] = board[i][j]
                        board[i][j] = 100


# 행렬 전치
def transpose():
    t_board = [[0 for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for j in range(n):
            t_board[n-1-j][i] = board[i][j]

    return t_board


if __name__ == '__main__':
    go_x = [0, 0, 1, -1]
    go_y = [1, -1, 0, 0]
    ans = 0
    n, m = map(int, input().split())
    board = []
    for _ in range(n):
        board.append(list(map(int, input().split())))

    while 1:
        # 크기가 가장 큰 블록 그룹을 찾는다.
        b_groups = search_block(board)
        if len(b_groups) == 0:
            break

        if len(b_groups) > 1:
            b_groups = get_rainbow(b_groups)

            if len(b_groups) > 1:
                b_groups = big_row_block(b_groups)

                if len(b_groups) > 1:
                    b_groups = big_col_block(b_groups)

        # 2. 1에서 찾은 블록 그룹의 모든 블록을 제거
        ans += remove_block(b_groups[0])

        # 3. 격자에 중력이 작용
        gravity()

        # 4. 격자가 90도 반시계 방향으로 회전
        board = transpose()

        # 5. 다시 격자에 중력이 작용
        gravity()

    print(ans)
